# Template Experiment notebook
This notebook walks you through how to read, add and modify experiments.

# Table of contents

TODO: Expand

## 0. Set-up and necessary requirements

TODO: Explain the set-up and requirements

In [ ]:
!pip uninstall NeuralNetworksTrainingPackage --y
#%pip uninstall TabularExperimentTrackerClient --y
!pip install git+https://github.com/DanielWarfield1/TabularExperimentTrackerClient
!pip install git+https://github.com/Bartosz-G/NeuralNetworksTrainingPackage

In [ ]:
import numpy as np
import pandas as pd
import sklearn
import torch
import time

In [ ]:
ex = ExperimentOrchstratorClient()
ex.get_credentials

# 1. Defining the experiment

## 1.1 Defining hyperparameter space

In this example we'll create 2 models:
- One sklearn model
- One Pytorch model

We begin by creating the possible space of hyperparameters and parameters which will later be used to build our models. You can subdivide your models by tasks or run one model for multiple tasks, but be sure to correctly assign model-task pairs in section 1.3.

In [ ]:
our_sklearn_model_space = {'param_1': {"distribution": "int_uniform", "min":1, "max":11},
                           'param_2': {"distribution": "float_uniform", "min":0.5, "max":1}}

our_pytorch_regression_model_space = {'param_1': {'distribution': 'constant', 'value': 0.1},
                                      'param_2': {'distribution': 'categorical', 'values':[10, 10, 15, 20]}}

our_pytorch_classification_model_space = {'param_1': {'distribution': 'constant', 'value': 0.1},
                                         'param_2': {'distribution': 'categorical', 'values':[10, 10, 15, 20]}}

Possible distributions:
- TODO: list all distributions

Remember that your further code should be able to re-create the models and training based on only the hyperparemeter space, so we also recommend including parameters like 'seed', 'cuda' and 'task'.

### Example:

In [ ]:
XGBoost_space = {
    "max_depth": {"distribution": "int_uniform", "min":1, "max":11},
    "n_estimators": {"distribution": "int_uniform", "min":100, "max":200},
    "min_child_weight": {"distribution": "log_uniform", "min":1, "max":1e2},
    "subsample": {"distribution": "float_uniform", "min":0.5, "max":1},
    "colsample_bylevel": {"distribution": "float_uniform", "min":0.5, "max":1},
    "colsample_bytree": {"distribution": "float_uniform", "min":0.5, "max":1},
    "gamma": {"distribution": "log_uniform", "min":1e-8, "max":7},
    "reg_lambda": {"distribution": "log_uniform", "min":1, "max":4},
    "reg_alpha": {"distribution": "log_uniform", "min":1e-8, "max":1e2},
}

LCN_reg_SGD_space = {
    'depth': {'distribution': 'int_uniform', 'min':1, 'max':11},
    'seed': {'distribution': 'constant', 'value': 42},
    'drop_type': {'distribution': 'categorical', 'values':['node_dropconnect', 'none']},
    'p': {'distribution': 'float_uniform', 'min':0.25, 'max':0.75},
    'ensemble_n': {'distribution': 'constant', 'value': 1},
    'shrinkage': {'distribution': 'constant', 'value': 1},
    'back_n': {'distribution': 'categorical', 'values':[0, 0, 0, 1]},
    'net_type': {'distribution': 'constant', 'value': 'locally_constant'},
    'hidden_dim': {'distribution': 'constant', 'value': 1},
    'anneal': {'distribution': 'categorical', 'values':['interpolation', 'none', 'approx']},
    'optimizer': {'distribution': 'constant', 'value': 'SGD'},
    'batch_size': {'distribution': 'categorical', 'values':[16,32,64,64,64,128,256]},
    'epochs': {'distribution': 'constant', 'value': 30},
    'lr': {'distribution': 'log_uniform', 'min':0.05, 'max':0.2},
    'momentum': {'distribution': 'constant', 'value': 0.9},
    'no_cuda': {'distribution': 'constant', 'value': False},
    'lr_step_size': {'distribution': 'categorical', 'values':[10, 10, 15, 20]},
    'gamma': {'distribution': 'constant', 'value': 0.1},
    'task': {'distribution': 'constant', 'value': 'regression'}
}

## 1.2 Assigning distributions to their models:

In [ ]:
model_groups = {
    'our_sklearn_model':{'model':'our_sklearn_model', 'hype':our_sklearn_model_space},
    'our_pytorch_regression_model':{'model':'our_pytorch_regression_model', 'hype':our_pytorch_regression_model_space},
    'our_pytorch_classification_model':{'model':'our_pytorch_classification_model', 'hype':our_pytorch_regression_model_space},
}

ex.def_model_groups(model_groups)

# 1.3 Assign experiments to appropriate tasks

TODO Explain:
- opml_reg_purnum_group:
- opml_reg_numcat_group:
- opml_class_purnum_group:
- opml_class_numcat_group:

In [ ]:
regression_models = {'our_sklearn_model':{'model':'our_sklearn_model', 'hype':our_sklearn_model_space},
                     'our_pytorch_regression_model':{'model':'our_pytorch_regression_model', 'hype':our_pytorch_regression_model_space}}

classification_models = {
    'our_sklearn_model':{'model':'our_sklearn_model', 'hype':our_sklearn_model_space},
    'our_pytorch_classification_model':{'model':'our_pytorch_classification_model', 'hype':our_pytorch_regression_model_space}}


applications = {'opml_reg_purnum_group': regression_models,
                'opml_reg_numcat_group': regression_models,
                'opml_class_purnum_group': classification_models,
                'opml_class_numcat_group': classification_models}

ex.def_applications(applications)

## 1.4 Registering an Experiment

In [ ]:
experiment_name = 'template_experiment'
ex.reg_experiment(experiment_name)
exp_info = ex.experiment_info()
successful_runs = exp_info['successful_runs']
required_runs = exp_info['required_runs']

# 2. Task pre-processing steps

In our experiments data pre-processing steps are applied right after downloading datasets in the main loop of the experiment. Object of class `dataPreProcessingEventEmitter()` stores all of the pre-processing steps in queues and is responsible for executing them in the right order and on the right partitions of data (such as on `train`). In our code we separate queues into task-specific queues (under namespace such as `regression`) and model-specific queues (under namespace such as `'our_pytorch_classification_model'`), although you're not limited to only those and can introduce your own logic.

You can also view this process as event-driven, where the `dataPreProcessingEventEmitter()` object listens for which model and task is coming in at every loop and responds by applying the correct transformations to the data.


### 2.1 Defining task specific pre-processing steps

Firstly, we begin by initialising task pre-processing steps stored in `NeuralNetworksTrainingPackage.dataprocessing.basic_pre_processing` module. Task pre-processing steps come in form of classes. When initialised those objects store parameters of how data pre-processing steps should be applied.

In [ ]:
from NeuralNetworksTrainingPackage.dataprocessing.basic_pre_processing import filterCardinality, quantileTransform, trunctuateData, oneHotEncodePredictors, oneHotEncodeTargets, toDataFrame, splitTrainValTest

n_sample = 20000
split = [0.5, 0.25, 0.25]
quantile_transform_distribution='uniform'

filter_cardinality = filterCardinality(transform = 'all')
trunctuate_data = trunctuateData(n = n_sample, transform = 'all')
one_hot_encode_predictors = oneHotEncodePredictors(transform = 'all')
one_hot_encode_targets = oneHotEncodeTargets(transform = 'all')
to_data_frame = toDataFrame(transform = 'all')
split_train_val_test = splitTrainValTest(split = split) # Special transformation
quantile_transform = quantileTransform(output_distribution = quantile_transform_distribution, transform = 'train')

Pre-processing steps can be divided into two, ordinary and special.

Ordinary steps:
- `trunctuateData(n, seed = None, transform = 'all')`
- `filterCardinality(transform = 'all')`
- `quantileTransform(n_quantiles=1000, output_distribution='uniform',
                 ignore_implicit_zeros=False,
                 subsample=10000,
                 random_state=None,
                 copy=True,
                 transform = 'all')`
- `toDataFrame(transform = 'all')`
- `oneHotEncodePredictors( transform = 'all')`
- `oneHotEncodeTargets(transform = 'all'):`

Ordinary classes all come with `transform = 'all'` parameter. This parameter dictates whether the transformation should be applied to the entire dataset `'all'`, only training `'train'`, only validation`'val'` or only test `'test'`. Bear in mind `transform = 'train'/'val'/'test'` is only available after the `splitTrainValTest` special step, before that step transform should be set to it's default `'all'`

Special steps:
- `splitTrainValTest(split = [0.5, 0.25, 0.25])`
- `toPyTorchDatasets(wrapper = CustomDataset):`

`splitTrainValTest(split = [train, val, test])`, is responsible for splitting the dataset into the train, validation and test sets. `toPyTorchDatasets(wrapper = CustomDataset)` is responsible for wrapping datasets into a pytorch friendly dataset format. Bear in mind after calling `toPyTorchDatasets`, no other pre-processing steps can be called. More on that in section 3.


### 2.2 Adding pre-processing steps to a queue

Next we need to add objects we've initiated into their corresponding tasks. `dataPreProcessingEventEmitter` works like a standard queue which sequentially applies all operations stored in that namespace, in our case `regression` or `classification`. Feel free to apply different pre-processing steps to different tasks, or add your own .

In [ ]:
from NeuralNetworksTrainingPackage.event_handler import dataPreProcessingEventEmitter

data_pre_processing = dataPreProcessingEventEmitter()

# Transformations will be applied in the order they're added to data_pre_processing
data_pre_processing.add_pre_processing_step('regression', filter_cardinality)
data_pre_processing.add_pre_processing_step('regression', trunctuate_data)
data_pre_processing.add_pre_processing_step('regression', one_hot_encode_predictors)
data_pre_processing.add_pre_processing_step('regression', to_data_frame)
data_pre_processing.add_pre_processing_step('regression', split_train_val_test)
data_pre_processing.add_pre_processing_step('regression', quantile_transform)


data_pre_processing.add_pre_processing_step('classification', filter_cardinality)
data_pre_processing.add_pre_processing_step('classification', trunctuate_data)
data_pre_processing.add_pre_processing_step('classification', one_hot_encode_predictors)
data_pre_processing.add_pre_processing_step('classification', one_hot_encode_targets) # different steps can be applied to different tasks
data_pre_processing.add_pre_processing_step('classification', to_data_frame)
data_pre_processing.add_pre_processing_step('classification', split_train_val_test)
data_pre_processing.add_pre_processing_step('classification', quantile_transform)

### 2.3 Adding your own pre-processing steps

If you'd like to add your own pre-processing steps using the following code template. The pre-processing step requires to have `self.seed = None`, `self.parent = None`, and `self.transform = transform`. The last one needs to be passed as an argument when initialising the object. The transformation steps are meant to receive X as `pd.DataFrame` and y as either `pd.DataFrame` or `pd.Series`, categorical_indicator as `List[bool]` and attribute_names as `List[str]`, and they're required to return them in the same format. You need to ensure proper type handling yourself! Transformation steps should be stateless. Although every step has a `transform` attribute you don't need to manually code separate transformations for `train`, `val`, `test`, the `dataPreProcessingEventEmitter` handles calling transformation steps on the appropriate partitions of the data.

In [ ]:
class exampleTransformationTemplate():
    def __init__(self, transform = 'all'): # You can add more arguments
        self.seed = None
        self.parent = None
        self.transform = transform
        # you can add more

    def apply(self, X, y, categorical_indicator, attribute_names):
        if not self.seed:
            self.seed = self.parent.seed

        # ---------
        # Add your own code here
        # Remember to adjust categorical_indicator as well as attribute_names if your transformation adjusts them
        # ---------

        return X, y, categorical_indicator, attribute_names

# 3. Model specific transformations

After task specific data transformations we can add model specific transformations. We're not required to add model specific transformations, if an event emitter doesn't find any model names, it does nothing. Remember to add them to the same event emitter as the task specific ones'!

In [ ]:
from NeuralNetworksTrainingPackage.dataprocessing.basic_pre_processing import CustomDataset, toPyTorchDatasets

to_pytorch_datasets = toPyTorchDatasets(wrapper = CustomDataset)

# Transformations will be called after general pre-processing steps, and in order they're added
data_pre_processing.add_pre_processing_step('our_pytorch_regression_model', to_pytorch_datasets)

data_pre_processing.add_pre_processing_step('our_pytorch_classification_model', to_pytorch_datasets)

# If our models don't require any additional pre-processing steps, you shouldn't add anything

`toPyTorchDatasets(wrapper = CustomDataset)` is responsible for wrapping datasets into a pytorch friendly dataset format. Bear in mind after calling `toPyTorchDatasets`, no other pre-processing steps can be called. By default, when indexed, the `CustomDataset` returns a tuple of tensors `torch.Size([1, number_of_predictor_columns])` and `torch.Size([1, number_of_outcome_columns])`, of `torch.float` data type. `CustomDataset` can be directly passed to `torch.utils.data.DataLoader()`. If your models requires different dimensions you can create your own pytorch dataset object and pass it as a wrapper to `toPyTorchDatasets`, skeleton code below.

### 3.1 Creating your own format of output

The Dataset object has the same requirements as a standard pytorch dataset object, but there're 2 additional constraints:
- has to take in specifically X, y, categorical_indicator, attribute_names
- has to have a `get_dims` methods that returns a number of predictor columns and output columns in form of a dict

You can find more on creating your own dataset object in pytorch's official documentation:
https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, X, Y, categorical_indicator, attribute_names, tensor_type=torch.float):
        assert isinstance(X, pd.DataFrame), "X must be a Pandas DataFrame"
        assert isinstance(Y, pd.DataFrame), "Y must be a Pandas DataFrame"

        # ---------
        # Add your own code here
        # ---------

    def get_dims(self):

        # ---------
        # Add your own code here
        # ---------

        return {'input_dim': num_columns_X, 'output_dim':num_columns_Y}

    def __len__(self):

        # ---------
        # Add your own code here
        # ---------

        return

    def __getitem__(self, idx):

        # ---------
        # Add your own code here
        # ---------

        return

### 3.2 How data transformations will be applied

The `data_pre_processing` object from `dataPreProcessingEventEmitter` class, has several methods:
- `.add_pre_processing_step('event_name', transformation_object)` to add a new transformation step object under that event name
- `.set_seef_for_all(seed)` allows for setting the same seed for all transformations which require one
- `.set_dataset(X, y, categorical_indicator, attribute_names)` which is how you pass it the datasets
- `.apply('event_name')` applies all of the transformations defined under the event name
- `.get_train_val_test()` retrieves train, val, and test datasets from the data_pre_processing event
- `.reset(hard = False)` reset the dataset (called automatically when calling `.set_dataset`)
 - `.reset(hard = True)` to reset everything including the seed and all the added transformation steps
- `.get('name')` retrieves a specific split of the data, name can be `train`, `val`, `test`

When `.apply('event_name')` doesn't find any `'event_name'` it does nothing.

In [ ]:
# Example
data_pre_processing.set_seed_for_all(seed)
data_pre_processing.set_dataset(X, y, categorical_indicator, attribute_names)
data_pre_processing.apply('regression')
data_pre_processing.apply('our_pytorch_regression_model')
train_data, val_data, test_data = data_pre_processing.get_train_val_test()

By default, if `toPyTorchDatasets(wrapper = CustomDataset)` is not applied, train_data, val_data, test_data, come in as a tuple of the corresponding `(X, y, categorical_indicator, attribute_names)`, if `toDataFrame` is included in the steps, both X, y are assured to be `pandas.DataFrame`. The `one_hot_encode_targets` encodes y with one-hot-encoding, so for binary classification it will have 2 columns or n columns for n-multi-label-classification.

# 4. Model Metrics

Next we need to add how our metrics will be calculated. Due to different models outputting different formats, you need to code in your own class  that will be responsible for calculating metrics. You're also free to add more metrics. Even if you're using different model for different tasks you still need to add them under `model_name` and `task`. `metric_model_pairs` dictionary should be available under the global namespace.

In [ ]:
our_sklearn_metrics_for_both_regression_and_classification = ourSklearnMetricsForBothRegressionAndClassification()
our_pytorch_regression_metrics = ourPytorchRegressionMetrics()
our_pytorch_classification_metrics = ourPytorchClassificationMetrics()


sklearn_metrics = {'regression': our_sklearn_metrics_for_both_regression_and_classification,
                   'classification': our_sklearn_metrics_for_both_regression_and_classification}

pytorch_regression_metrics = {'regression': our_pytorch_regression_metrics}

pytorch_classification_metrics = {'classification': our_pytorch_classification_metrics}

metric_model_pairs = {
    'our_sklearn_model': sklearn_metrics,
    'our_pytorch_regression_model': pytorch_classification_metrics,
    'our_pytorch_classification_model': pytorch_regression_metrics

}

### 4.1 Adding your own sklearn metrics class

TODO: create a unified sklearn metrics calculator

In [ ]:
class MySklearnModelMetrics():
    def __init__(self):
        # You can add initialisation parameters if you wish to control how metrics are gathered at run-time
        pass


    def get_metrics(self, model, test_data, hyperparameters, test_set_name=None):
        if test_set_name:
            assert isinstance(test_set_name, str), "test_set_name must be a string, such as train, val, test"

        """
        Input:
            model: your model
            test_loader: pytorch DataLoader, returns tensors of batch_size x num_columns from dataset wrapper initialised in section 3.
            hyperparameters: your hyperparameters initialised in section 1.
            test_set_name: str appended to the key's of the dictionary returned at the end
        """

        # ---------
        # Add your own code here
        # ---------

        metrics = {
            'accuracy_score': accuracy,
            'roc_auc_score': roc_auc,
            'confusion_matrix': confusion_matrix # Add more metrics here if you wish to gather more
        }

        if test_set_name:
            metrics_name = f'{test_set_name}_metrics'
            final_metrics = {metrics_name: metrics}
        else:
            final_metrics = {'metrics': metrics}

        return final_metrics

    """
    Example output for classification:
        {'train_metrics': {
            'accuracy_score': 0.59045,
            'roc_auc_score': 0.023819148540496826,
            'confusion_matrix': [[521, 14],[16, 312]]
            }
        }
    """

    """
    Example output for regression:
    {'train_metrics': {'RMSE': 0.5904541168055158,
                       'r2_score': 0.023819148540496826,
                       'se_quant': {0.01: 7.687292236369103e-05, 0.025: 0.00033428650931455195, 0.05: 0.0007752738310955465
                                    0.1: 0.003265600185841322, 0.2: 0.008951567113399506, (...), 0.99: 1.9776358604431152
                       }
     }
    """



### 4.2 Adding your own pytorch metrics class

Metrics class is required to have `get_metrics()`, and `get_all()` methods. Only difference is `get_all()` outputs both metrics and a loss while `get_metrics()` outputs only metrics. This was done to avoid calculating train_loss twice, both during training and after the training. If computational power is not of concern, you can implement only one of them and adjust the main training loop to only use one of them. By default we gather

In [ ]:
class MyPytorchModelMetrics():
    def __init__(self):
        # You can add initialisation parameters if you wish to control how metrics are gathered at run-time
        pass


    def get_metrics(self, model, test_loader, hyperparameters: dict, test_set_name=None):
        if test_set_name:
            assert isinstance(test_set_name, str), "test_set_name must be a string, such as train, val, test"

        """
        Input:
            model: your model
            test_loader: pytorch DataLoader, returns tensors of batch_size x num_columns from dataset wrapper initialised in section 3.
            hyperparameters: your hyperparameters initialised in section 1.
            test_set_name: str appended to the key's of the dictionary returned at the end
        """

        # ---------
        # Add your own code here
        # ---------

        metrics = {
            'accuracy_score': accuracy,
            'roc_auc_score': roc_auc,
            'confusion_matrix': confusion_matrix # Add more metrics here if you wish to gather more
        }

        if test_set_name:
            metrics_name = f'{test_set_name}_metrics'
            final_metrics = {metrics_name: metrics}
        else:
            final_metrics = {'metrics': metrics}

        return final_metrics

    """
    Example output for classification:
        {'train_metrics': {
            'accuracy_score': 0.59045,
            'roc_auc_score': 0.023819148540496826,
            'confusion_matrix': [[521, 14],[16, 312]]
            }
        }
    """

    """
    Example output for regression:
    {'train_metrics': {'RMSE': 0.5904541168055158,
                       'r2_score': 0.023819148540496826,
                       'se_quant': {0.01: 7.687292236369103e-05, 0.025: 0.00033428650931455195, 0.05: 0.0007752738310955465
                                    0.1: 0.003265600185841322, 0.2: 0.008951567113399506, (...), 0.99: 1.9776358604431152
                       }
     }
    """



    def get_all(self, model, test_loader, hyperparameters:dict, test_set_name=None):
        if test_set_name:
            assert isinstance(test_set_name, str), "test_set_name must be a string, such as train, val, test"

        # ---------
        # Add your own code here
        # ---------

        metrics = {
            'accuracy_score': accuracy,
            'roc_auc_score': roc_auc,
            'confusion_matrix': confusion_matrix}

        if test_set_name:
            metrics_name = f'{test_set_name}_metrics'
            final_metrics = {metrics_name: metrics}
        else:
            final_metrics = {'metrics': metrics}


        return final_metrics



    """
    Example output for classification:
        {'train_loss': 0.357,
        'train_metrics': {
            'accuracy_score': 0.59045,
            'roc_auc_score': 0.023819148540496826,
            'confusion_matrix': [[521, 14],[16, 312]]
            }
        }
    """

    """
    Example output for regression:
    {'train_loss': 0.35755656490325927,
     'train_metrics': {'RMSE': 0.5904541168055158,
                       'r2_score': 0.023819148540496826,
                       'se_quant': {0.01: 7.687292236369103e-05, 0.025: 0.00033428650931455195, 0.05: 0.0007752738310955465
                                    0.1: 0.003265600185841322, 0.2: 0.008951567113399506, (...), 0.99: 1.9776358604431152
                       }
     }
    """

Additionally, we've implemented `ConfusionMatrix()` in the `metrics/LcnMetrics.py` file that can simplify your confusion matrix calculations:

In [ ]:
from metrics.LcnMetrics import ConfusionMatrix

preds_1 = torch.tensor([1, 0, 0, 1, 1])
target_1 = torch.tensor([1, 0, 1, 0, 1])

preds_2 = torch.tensor([0, 0, 0, 1, 1])
target_2 = torch.tensor([0, 0, 1, 0, 1])

preds_3 = torch.tensor([0, 0, 1])
target_3 = torch.tensor([0, 0, 1])

confusion_accumulator = ConfusionMatrix()

confusion_accumulator.update(preds_1, target_1)
confusion_accumulator.update(preds_2, target_2)
confusion_accumulator.update(preds_3, target_3)

confusion_matrix = confusion_accumulator.compute()
confusion_matrix = [[int(i) for i in row] for row in confusion_matrix.cpu().numpy().tolist()]
# confusion_matrix: [[5, 3],[2, 4]]

# 5. Model Training routine

Finally we need to add `init_model` function, and `ModelTrainingRoutine` class (training object will be initialised in the main training loop).

In [ ]:
sklearn_model_and_training = {'model_init':init_Sklearn_Model, 'training_routine': SklearnTrainingRoutine}

pytorch_regression_model_and_training = {'model_init':init_Pytorch_Reg_Model, 'training_routine': PytorchRegTrainingRoutine}

pytorch_cls_model_and_training = {'model_init':init_Pytorch_Cls_Model, 'training_routine': PytorchClsTrainingRoutine}

model_training_pairs = {
    'our_sklearn_model': sklearn_model_and_training,
    'our_pytorch_regression_model': pytorch_regression_model_and_training,
    'our_pytorch_classification_model': pytorch_cls_model_and_training
}

### 5.1 Creating your own init

Init function should take in unpacked hyperparameters and return an appropriate model. The output of `init_model` function will then be directly passed to `ModelTrainingRoutine` object

In [ ]:
def init_Model(**hyperparameters):

    # ---------
    # Add your own code here
    # ---------

    return # Model

### 5.2 Adding your own sklearn training routine

In [ ]:
class MySklearnModelTrainingRoutine():
    def __init__(self, all_hyperparameters):
        # Assign proper hyperparameters to arguments
        # self.my_hyperparam_1 = my_hyperparam_1
        # self.my_hyperparam_2 = my_hyperparam_2

    def train(self, model, train_data):
        # Input comes as a tuple of data, target, categorical_indicator, attribute_names

        X, y, categorical_indicator, attribute_names = train_data

        return model


### 5.3 Adding your own pytorch training routine

In [ ]:
class MyPytorchModelTrainingRoutine():
    def __init__(self, all_hyperparameters):
        # Assign proper hyperparameters to arguments
        # self.my_hyperparam_1 = my_hyperparam_1
        # self.my_hyperparam_2 = my_hyperparam_2


        self.optimizer = None # Required, must be None at the start
        self.scheduler = None # Required, must be None at the start

        self.epoch = 0 # Required, must be 0 at the start



    def set_optimizer_scheduler(self, model):
        self.optimizer = # Your code here
        self.scheduler = # Your code here, not required

    def scheduler_step(self, epoch):
        if self.scheduler is not None:
            self.scheduler.step(epoch)

        self.epoch = epoch


    def train(self, model, train_loader):
        model.train()

        """
        Input:
            model: your model
            train_loader: pytorch DataLoader, returns tensors of batch_size x num_columns from dataset wrapper initialised in section 3.
        """

        # ---------
        # Add your own code here
        # ---------

        # =======
        # We recommend estimating the train loss during training time
        # If the loss becomes NaN, train_loss output should be set to None to, this will trigger early stopping but still
        # log the train_loss to the orchestrator
        # =======

        final_metric = {'train_loss': train_loss}
        return train_loss


# 6. Main Experiment Loop

In [ ]:
torch_models = ('our_pytorch_regression_model',)

sklearn_models = ('our_sklearn_model')

num_of_experiment_to_be_run = # modify to appropriate

for i in range(num_of_experiment_to_be_run):
    print(f'==== Begin run:{i} ====')
    run_info = ex.begin_run_sticky()

    hyperparameters = run_info['hyp']
    model_name = run_info['model']

    task = hyperparameters['task']
    seed = hyperparameters['seed']


    torch.manual_seed(seed)
    np.random.seed(seed)


    print('---- Loading datasets ----')
    X, y, categorical_indicator, attribute_names = ex.opml_load_task(run_info['mtpair_task'])

    # Pre-processing
    data_pre_processing.set_seed_for_all(seed)
    data_pre_processing.set_dataset(X, y, categorical_indicator, attribute_names)
    data_pre_processing.apply(task)
    data_pre_processing.apply(model_name)
    train_data, val_data, test_data = data_pre_processing.get_train_val_test()


    # Getting appropriate metrics
    metrics_calculator = metric_model_pairs[model_name][task]


    match model_name:
        case _ if model_name in sklearn_models:
            init_model = model_training_pairs[model_name]['model_init']
            TrainingRoutine = model_training_pairs[model_name]['training_routine']

            model = init_model(**hyperparameters)
            training_routine = TrainingRoutine(**hyperparameters)

            start_time = time.time()
            fit_model = training_routine.train(model, train_data)

            training_time = time.time()-start_time
            train_all_metrics = metrics_calculator.get_all(model, train_data, hyperparameters, 'train')
            val_all_metrics = metrics_calculator.get_all(model, val_data, hyperparameters, 'val')
            test_all_metrics = metrics_calculator.get_all(model, test_data, hyperparameters, 'test')

            final_metrics = {}
            final_metrics.update(train_all_metrics)
            final_metrics.update(val_all_metrics)
            final_metrics.update(test_all_metrics)
            final_metrics.update({'epoch': None})
            final_metrics.update({'epoch_time': training_time})

            ex.update_run(final_metrics)



        case _ if model_name in torch_models:
            # hyperparameters will be updated with {'input_dim': num_columns_X, 'output_dim':num_columns_Y}
            # this is required for proper initialisation of some models
            hyperparameters.update(train_data.get_dims())


            train_batch_size = hyperparameters['batch_size']
            train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=train_batch_size,shuffle= True)
            val_dataloader = torch.utils.data.DataLoader(val_data,batch_size=len(val_data),shuffle= True)
            test_dataloader = torch.utils.data.DataLoader(test_data,batch_size=len(test_data),shuffle= True)


            init_model = model_training_pairs[model_name]['model_init']
            TrainingRoutine = model_training_pairs[model_name]['training_routine']


            model = init_model(**hyperparameters)
            training_routine = TrainingRoutine(**hyperparameters)

            training_routine.set_optimizer_scheduler(model)

            start_epoch = 1  # start from epoch 1 or last checkpoint epoch
            total_epochs = hyperparameters['epochs']
            start_time = time.time()

            for epoch in range(start_epoch, total_epochs + start_epoch):
                print(f"----{epoch}th training epoch ----")
                epoch_metrics = {}

                training_routine.scheduler_step(epoch)
                train_loss = training_routine.train(model, train_dataloader)


                if train_loss is None:
                    print('---Stopping training due to loss being nan!---')
                    epoch_metrics = {'train_loss': train_loss, 'epoch': epoch}
                    ex.update_run(epoch_metrics)
                    break

                if epoch == total_epochs:
                    continue

                epoch_metrics.update(train_loss)
                epoch_metrics.update({'epoch': epoch})
                ex.update_run(epoch_metrics)
                print(epoch_metrics)

            else:
                final_metrics = {}
                training_time = time.time()-start_time

                train_metrics = metrics_calculator.get_metrics(model, train_dataloader, hyperparameters, 'train')
                val_all_metrics = metrics_calculator.get_all(model, val_dataloader, hyperparameters, 'val')
                test_all_metrics = metrics_calculator.get_all(model, test_dataloader, hyperparameters, 'test')

                final_metrics.update(train_loss)
                final_metrics.update(train_metrics)
                final_metrics.update(val_all_metrics)
                final_metrics.update(test_all_metrics)
                final_metrics.update({'epoch': epoch})
                final_metrics.update({'epoch_time': training_time})

                ex.update_run(final_metrics)
                print(final_metrics)

    ex.end_run()